## Introduction

This notebook showcases the power of BigQuery object tables and multimodal Gemini models for image analysis. We'll explore how these technologies can be used to automate the monitoring of physical assets.

Our example focuses on a common challenge for municipalities: maintaining bus stops. We'll demonstrate how to analyze pictures of bus stops to determine their state of repair. This could involve identifying issues like broken benches, damaged shelters, or excessive litter.

Here's how we envision this working:

* Buses are equipped with cameras that capture photos of each bus stop they pass.
* At the end of the day, these photos are uploaded to a central repository.
* A data pipeline processes the pictures, using Gemini to detect anomalies and flag potential issues.
* Maintenance crews can then be dispatched to address the identified problems.

While we're using bus stops as our example, the techniques demonstrated here can be readily adapted to other asset monitoring scenarios across various industries.

Let's get started with the code!

## Initializing the environment

First, let's create some environment variables:

In [ ]:
import os
PROJECT_ID = os.environ['GOOGLE_CLOUD_PROJECT']
SOURCE_PATH = f"gs://bus-stop-images/julien/notebook-sources"
TARGET_PATH = f"gs://{PROJECT_ID}-multimodal/target"
REGION = "us-central1"

### Utility functions

Define some utility functions that we'll use throughout this notebook. Refer to the comments in the functions to get more details about what they do.

In [ ]:
import re
import base64
from typing import List, Dict, Union
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
from IPython.display import display, HTML


# Don't truncate strings in BigQuery query results
pd.set_option('display.max_colwidth', None)


def parse_gcs_uri(uri: str) -> tuple[str, str]:
    """
    Parse GCS URI of the form gs://bucket or gs://bucket/path/to/file
    """
    uri = uri.rstrip('/')
    match = re.match(r'gs://([^/]+)(?:/(.*))?', uri)
    if not match:
        raise ValueError(f"Invalid GCS URI format: {uri}")
    bucket_name, path = match.groups()
    return bucket_name, path or ''


def list_gcs_contents(uri: str):
    """
    Lists contents of a GCS bucket/path
    """
    bucket_name, prefix = parse_gcs_uri(uri)
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blobs = bucket.list_blobs(prefix=prefix)

    for blob in blobs:
        print(blob.name)


def show_object_table(table):
    """
    Shows the contents of the given BigQuery object table
    """
    query = f"""
    SELECT * FROM `{table}`
    ORDER BY updated;
    """
    df = run_bigquery(query).to_dataframe()
    return HTML(df[['metadata', 'uri']].to_html(escape=False))


def run_bigquery(query: str):
    """
    Runs the provided BigQuery query and returns the result
    """
    job_config = bigquery.QueryJobConfig()
    bq_client = bigquery.Client()
    query_job = bq_client.query(query, job_config=job_config)
    return query_job.result()


def get_image_from_gcs(uri: str) -> str:
    """Fetches an image from Google Cloud Storage as an HTML img tag.

    Args:
        uri: Full GCS URI in the format 'gs://bucket-name/path/to/image.jpg'

    Returns:
        HTML img tag with base64-encoded image data, or None if the image
        could not be fetched.
    """
    try:
        bucket_name, blob_name = parse_gcs_uri(uri)

        # Get image bytes
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        image_bytes = blob.download_as_bytes()

        # Convert to base64
        base64_image = base64.b64encode(image_bytes).decode('utf-8')
        return f'<img src="data:image/jpg;base64,{base64_image}" width="300">'

    except Exception as e:
        print(f"Error fetching image from GCS: {e}")
        return None


def preview_batch_images(batch: List[Dict[str, str]], source: str):
   container_id = f"image-container-{hash(str(batch))}"
   display(HTML(f"""
       <div id="{container_id}" style="display: flex; flex-wrap: wrap; gap: 10px; justify-content: flex-start;">
       </div>
   """))
   
   for i, item in enumerate(batch):
       try:
           gcs_uri = f"{source}/{item['path']}"
           img_tag = get_image_from_gcs(gcs_uri)
           if img_tag:
               wrapper_id = f"{container_id}-wrapper-{i}"
               display(HTML(f"""
                   <div id="{wrapper_id}" style="text-align: center;">
                       {img_tag}
                       <div style="margin-top: 5px;">{item['path']}</div>
                   </div>
                   <script>
                       document.getElementById('{container_id}').appendChild(
                           document.getElementById('{wrapper_id}')
                       );
                   </script>
               """))
           else:
               print(f"Failed to load image: {item['path']}")
       except Exception as e:
           print(f"Error: {e}")


def upload_batch(
    batch: List[Dict[str, str]],
    source: str,
    target: str,
    batch_number: Union[int, str]
):
    """
    Copy images from source GCS location to target with metadata.
    """
    client = storage.Client()

    source_bucket_name, source_prefix = parse_gcs_uri(source)
    target_bucket_name, target_prefix = parse_gcs_uri(target)

    source_bucket = client.bucket(source_bucket_name)
    target_bucket = client.bucket(target_bucket_name)

    count = 0
    for item in batch:
        try:
            source_path = (
                f"{source_prefix}/{item['path']}" if source_prefix
                else item['path']
            )
            target_path = (
                f"{target_prefix}/{item['path']}" if target_prefix
                else item['path']
            )

            # Get source blob
            source_blob = source_bucket.blob(source_path)
            if not source_blob.exists():
                raise FileNotFoundError(f"Source file {source_path} not found")

            # Download the image bytes
            image_bytes = source_blob.download_as_bytes()

            # Prepare target blob with metadata
            target_blob = target_bucket.blob(target_path)
            target_blob.metadata = {
                "batch_number": f"batch-{batch_number}",
                **{k: str(v) for k, v in item.items() if k != "path"}
            }

            # Upload bytes
            target_blob.upload_from_string(
                image_bytes,
                content_type=source_blob.content_type
            )
            count += 1
            print(f"({count}/{len(batch)}) Uploaded: {item['path']}")

        except Exception as e:
            print("Error processing {item['path']}: {str(e)}")

def style_incident_row(row):
    """
    Custom styling function for bus stops incidents
    """
    if not row['resolved']:  # When resolved is False
        return ['background-color: #ffcccc'] * len(row)  # Light red for all columns
    return [''] * len(row)  # No styling for other cases


def show_incidents():
    df = run_bigquery(incidents_list_query).to_dataframe()
    df['open_report_image'] = df['open_report_gcs_uri'].apply(lambda uri: get_image_from_gcs(uri) if uri is not None else "")
    df['resolve_report_image'] = df['resolve_report_gcs_uri'].apply(lambda uri: get_image_from_gcs(uri) if uri is not None else "")
    
    styles = [
        {'selector': 'td.col_open_report_image img, td.col_resolve_report_image img', 
         'props': [('transition', 'transform 0.3s')]}
    ]
    
    styled_df = df[['bus_stop_id', 'resolved', 'open_report_description', 'open_report_image', 'resolve_report_description', 'resolve_report_image']].style\
        .set_table_styles(styles)\
        .set_table_attributes('style="transform-style: preserve-3d;"')\
        .apply(style_incident_row, axis=1)
    
    return HTML(styled_df.to_html(escape=False))

In [ ]:
# Note: This magic comes preinstalled in BigQuery notebooks
%load_ext bigquery_magics

In [ ]:
%%html
<style>
img:hover {
    transform: scale(2.5);
    z-index: 1000;
}
</style>

### Cloud Storage bucket and Cloud Resource connection

Create a Cloud Storage bucket:

In [ ]:
from google.cloud import storage
from google.api_core import exceptions
storage_client = storage.Client(project=PROJECT_ID)
bucket_name = f"{PROJECT_ID}-multimodal"
try:
    bucket = storage_client.create_bucket(bucket_name)
    print(f"Bucket {bucket_name} created")
except exceptions.Conflict:
    # Bucket already exists - return the existing bucket
    bucket = storage_client.bucket(bucket_name)
    print(f"Bucket {bucket_name} already exists")
except Exception as e:
    print(f"Error creating bucket {bucket_name}: {e}")

Enable some APIs:

In [ ]:
!gcloud services enable --project {PROJECT_ID} bigqueryconnection.googleapis.com bigquerystorage.googleapis.com

Create a Cloud Resource connection:

In [ ]:
!bq mk \
--connection \
--location={REGION} \
--project_id={PROJECT_ID} \
--connection_type=CLOUD_RESOURCE \
multimodal

### BigQuery tables

Create the objects table connected to the GCS bucket:

In [ ]:
run_bigquery(
f"""
CREATE OR REPLACE EXTERNAL TABLE `multimodal.objects`
WITH CONNECTION `{REGION}.multimodal`
OPTIONS(
  object_metadata = 'SIMPLE',
  uris = ['{TARGET_PATH}/*']
)
"""
)

Create the BigQuery tables that we'll use in the demo:

In [ ]:
%%bigquery

CREATE OR REPLACE TABLE `multimodal.reports` (
  report_id STRING,
  uri STRING,
  updated TIMESTAMP,
  bus_stop_id STRING,
  cleanliness_level INT64,
  safety_level INT64,
  description STRING
);

CREATE OR REPLACE TABLE `multimodal.incidents` (
  incident_id STRING NOT NULL,
  bus_stop_id STRING NOT NULL,
  resolved BOOL NOT NULL,
  open_report_id STRING,
  resolve_report_id STRING
);

CREATE OR REPLACE TABLE `multimodal.object_watermark` (
  process_time TIMESTAMP
);
INSERT INTO `multimodal.object_watermark` (SELECT TIMESTAMP('1970-01-01'));

CREATE OR REPLACE TABLE `multimodal.report_watermark` (
  process_time TIMESTAMP
);
INSERT INTO `multimodal.report_watermark` (SELECT TIMESTAMP('1970-01-01'));

## Uploading the first batch of pictures

Let's now look at the first batch of pictures. We specify the pictures' filenames as well as the corresponding bus stop IDs:

In [ ]:
batch_1 = [
    {"path": "QD-01.jpg", "stop_id": "QD"},
    {"path": "NA-01.jpg", "stop_id": "NA"},
    {"path": "QB-01.jpg", "stop_id": "QB"},
    {"path": "PA-02.jpg", "stop_id": "PA"},
    {"path": "RD-02-dead-leaves.jpg", "stop_id": "RD"},
    {"path": "PC-01.jpg", "stop_id": "PC"},
    {"path": "NC-02.jpg", "stop_id": "NC"},
    {"path": "MD-02.jpg", "stop_id": "MD"},
    {"path": "PB-01.jpg", "stop_id": "PB"},
    {"path": "ND-02-damaged.jpg", "stop_id": "ND"},
    {"path": "QA-02-dirty.jpg", "stop_id": "QA"},
    {"path": "SD-03-dirty-damaged.jpg", "stop_id": "SD"},
    {"path": "MC-02-dirty-damaged.jpg", "stop_id": "MC"},
    {"path": "MA-01.jpg", "stop_id": "MA"},
    {"path": "RC-02-crazy1.jpg", "stop_id": "RC"},
    {"path": "SC-02-broken-glass.jpg", "stop_id": "SC"},
    {"path": "UB-01.jpg", "stop_id": "UB"},
]

Preview the pictures:

In [ ]:
preview_batch_images(batch_1, SOURCE_PATH)

Upload the pictures to the bucket - the bus stop IDs are also attached to the files that land in the bucket:

In [ ]:
upload_batch(
    batch=batch_1,
    source=SOURCE_PATH,
    target=TARGET_PATH,
    batch_number=1
)

See the files that have just been uploaded to the bucket:

In [ ]:
list_gcs_contents(TARGET_PATH)

Let's see the contents of the object table that is linked to the bucket. You see the list of pictures as well as the bus stop IDs, which are stored as object metadata:

In [ ]:
show_object_table("multimodal.objects")

Now create a model to analyze the pictures:

In [ ]:
run_bigquery(
f"""
CREATE OR REPLACE MODEL
`multimodal.gemini_model`
REMOTE WITH CONNECTION `{REGION}.multimodal`
OPTIONS ( endpoint = 'gemini-1.5-flash-001' );
"""
)

Now let's look at the first main query of our data pipeline. This purpose of this query is to extract insights from the uploaded pictures. In particular, it prompts a Gemini image model to assess the state of all bus stops. The provided prompt asks Gemeni to rate the cleanliness and safety and to provide a detailed description of the scenes:

In [ ]:
reports_creation_query = """
DECLARE last_process_time TIMESTAMP;
DECLARE new_process_time TIMESTAMP;

BEGIN TRANSACTION;
SET last_process_time = (SELECT process_time FROM `multimodal.object_watermark`);
SET new_process_time = CURRENT_TIMESTAMP();

INSERT INTO `multimodal.reports`
SELECT
  GENERATE_UUID() as report_id,
  uri,
  updated,
  (SELECT value FROM UNNEST(metadata) WHERE name = 'stop_id') AS bus_stop_id,
  CAST (JSON_EXTRACT(ml_generate_text_llm_result, '$.cleanliness_level') AS INT64) AS cleanliness_level,
  CAST (JSON_EXTRACT(ml_generate_text_llm_result, '$.safety_level') AS INT64) AS safety_level,
  JSON_EXTRACT(ml_generate_text_llm_result, '$.description') AS description,
FROM
  ML.GENERATE_TEXT(
   MODEL `multimodal.gemini_model`,
   TABLE `multimodal.objects`,
   STRUCT (
   '''
Analyze this picture of a bus stop and provide details that might affect a person's decision to use the bus stop.
Return your answer in valid JSON format (without JSON decorators) with these fields:
* "cleanliness_level" (Integer): Rate the cleanliness of this bus stop. Possible values: 1="very dirty and warrants urgent cleaning", 2="clean or mildly clean". These are considered cleanliness issues: littering, broken glass, spillage of liquids/oil, ground covered in dead leaves, excessive dirt on benches that would prevent or detract from sitting, unsightly graffitis on the bus stop or surrounding structures, disfigured amenities, and anything that would be very unsightly or unreasonably dirty.
* "safety_level" (Integer): Rate the safety of this bus stop. Possible values: 1="unsafe and warrants attention", 2="safe and doesn't warrant attention". These are considered safety issues: broken glass, fallen tree/branches, fire, flooding, electrical hazard, ground covered in dead leaves, dangerous objects, damaged benches/shelters/amenities, slippery ground (e.g. presence of potentially slippery ice on the ground), icicles, low lighting at night, and anything else that could potentially represent a danger for people.
* "description" (String): Provide a detailed textual description (maximum of 120 words) of what is in the picture, with an emphasis on cleanliness and safety.
All fields are required.
   ''' AS prompt,
   0.2 AS temperature,
   TRUE AS flatten_json_output,
   8192 AS max_output_tokens)
  )
WHERE content_type = "image/jpeg" AND updated > last_process_time;

-- Update the process time watermark
UPDATE `multimodal.object_watermark`
SET process_time = new_process_time
WHERE TRUE;

COMMIT TRANSACTION;
"""

Let's run the query:

In [ ]:
run_bigquery(reports_creation_query)

Now let's look at the results in the reports table:

In [ ]:
%%bigquery
SELECT * FROM `multimodal.reports`;

In this table you can see:
* The list of pictures that were just uploaded and the corresponding stop IDs.
* The cleanliness and safety ratings that Gemini provided.
* Textual descriptions of what Gememi could see in the pictures.

Now let's look at the second query of our data pipeline.

This query parses the results from the reports and creates incidents as needed. If it sees that a bus stop was given low cleanliness or safety ratings, it will create a new incident in the `incidents` table with all the details relating to that situation. If, however, it finds high ratings for a bus stop that had previously been flagged, it will close the prior incident for that bus stop:

In [ ]:
incidents_update_query = """
DECLARE last_process_time TIMESTAMP;
DECLARE new_process_time TIMESTAMP;

BEGIN TRANSACTION;
SET last_process_time = (SELECT process_time FROM `multimodal.report_watermark`);
SET new_process_time = CURRENT_TIMESTAMP();

-- Main MERGE statement to update or insert incidents based on new reports
MERGE `multimodal.incidents` AS target
USING (
  -- CTE to get the latest report for each bus stop
  WITH latest_reports AS (
    SELECT
      *,
      -- Assign row numbers to reports for each bus stop, ordered by update time descending
      ROW_NUMBER() OVER (PARTITION BY bus_stop_id ORDER BY updated DESC) AS rn
    FROM `multimodal.reports`
    WHERE updated > last_process_time
  )
  -- Main subquery to prepare data for MERGE operation
  SELECT
    lr.bus_stop_id,
    -- Determine if an incident should be resolved based on cleanliness/safety threshold
    CASE
      WHEN lr.cleanliness_level >= 2 AND lr.safety_level >= 2 THEN TRUE
      ELSE FALSE
    END AS should_resolve,
    i.incident_id,
    lr.report_id,
    i.open_report_id
  FROM latest_reports lr
  -- Left join to find existing open incidents for each bus stop
  LEFT JOIN `multimodal.incidents` i
    ON lr.bus_stop_id = i.bus_stop_id
    AND i.resolved = FALSE
  -- Only consider the most recent report for each bus stop
  WHERE lr.rn = 1
) AS source
ON target.incident_id = source.incident_id

-- Update existing incidents: mark as resolved if cleanliness/safety has improved
WHEN MATCHED AND source.should_resolve THEN
  UPDATE SET
    resolved = TRUE,
    resolve_report_id = source.report_id

-- Insert new incidents: create for stops with low cleanliness/safety and no open incident
WHEN NOT MATCHED AND NOT source.should_resolve THEN
  INSERT (incident_id, bus_stop_id, resolved, open_report_id)
  VALUES (GENERATE_UUID(), source.bus_stop_id, FALSE, source.report_id)

-- Update existing incidents: set open_report_id if it's missing
WHEN MATCHED AND NOT source.should_resolve AND target.open_report_id IS NULL THEN
  UPDATE SET
    open_report_id = source.report_id;

-- Update the process time watermark
UPDATE `multimodal.report_watermark`
SET process_time = new_process_time
WHERE TRUE;

COMMIT TRANSACTION;
"""

Let's run the query:

In [ ]:
run_bigquery(incidents_update_query)

Let's now run a query to load and display all the created incidents:

In [ ]:
incidents_list_query = f"""
    WITH relevant_reports AS (
      SELECT DISTINCT r.uri
      FROM `multimodal.incidents` i
      LEFT JOIN `multimodal.reports` r ON i.open_report_id = r.report_id OR i.resolve_report_id = r.report_id
      WHERE r.uri IS NOT NULL
    ),
    gcs_uris AS (
      SELECT uri
      FROM `multimodal.objects`
      WHERE uri IN (SELECT uri FROM relevant_reports)
    )
    SELECT
      i.bus_stop_id,
      i.resolved,
      open_r.description AS open_report_description,
      open_su.uri AS open_report_gcs_uri,
      resolve_r.description AS resolve_report_description,
      resolve_su.uri AS resolve_report_gcs_uri
    FROM
      `multimodal.incidents` i
    LEFT JOIN
      `multimodal.reports` open_r ON i.open_report_id = open_r.report_id
    LEFT JOIN
      gcs_uris open_su ON open_r.uri = open_su.uri
    LEFT JOIN
      `multimodal.reports` resolve_r ON i.resolve_report_id = resolve_r.report_id
    LEFT JOIN
      gcs_uris resolve_su ON resolve_r.uri = resolve_su.uri
    ORDER BY
      i.resolved ASC,
      i.incident_id;
"""

In [ ]:
show_incidents()

In the above dashboard, you see that a number of incidents were reported for a variety of issues. The incidents show the pictures of the problematic bus stops as well as the corresponding textual description of the situations.

**TODO**: Explain what the user would be supposed to do with this dashboard (i.e. monitor issues, dispatch technicians on site to address the situations, etc.).

## Uploading the second batch

Now let's upload another batch of pictures:

In [ ]:
batch_2 = [
    {"path": "ND-02.jpg", "stop_id": "ND"},
    {"path": "MB-01-ice-snow.jpg", "stop_id": "MB"},
    {"path": "NC-01-dirty.jpg", "stop_id": "NC"},
    {"path": "MA-02.jpg", "stop_id": "MA"},
    {"path": "SC-02.jpg", "stop_id": "SC"},
    {"path": "PB-02.jpg", "stop_id": "PB"},
    {"path": "TA-01-graffitis.jpg", "stop_id": "TA"},
    {"path": "UA-01.jpg", "stop_id": "UA"},
    {"path": "UC-02-dirty.jpg", "stop_id": "UC"},
    {"path": "MC-02.jpg", "stop_id": "MC"},
    {"path": "NA-01-ice-snow.jpg", "stop_id": "NA"},
    {"path": "MA-02-broken-glass.jpg", "stop_id": "MA"},
    {"path": "UC-02.jpg", "stop_id": "UC"},
    {"path": "TC-02.jpg", "stop_id": "TC"},
    {"path": "NB-01.jpg", "stop_id": "NB"},
]

In [ ]:
preview_batch_images(batch_2, SOURCE_PATH)

In [ ]:
upload_batch(
    batch=batch_2,
    source=SOURCE_PATH,
    target=TARGET_PATH,
    batch_number=2
)

Create the reports for the newly uploaded pictures:

In [ ]:
run_bigquery(reports_creation_query)

Now update the incidents based on the new reports:

In [ ]:
run_bigquery(incidents_update_query)

See the updated list of incidents:

In [ ]:
show_incidents()

**TODO**: Explain what's in the dashboard. Some incidents have been resolved, some have not. There are also some new incidents. Etc.

## Uploading the third batch

Upload the third batch.

In [ ]:
batch_3 = [
    {"path": "QA-01.jpg", "stop_id": "QA"},
    {"path": "TD-02.jpg", "stop_id": "TD"},
    {"path": "QD-02.jpg", "stop_id": "QD"},
    {"path": "SB-02.jpg", "stop_id": "SB"},
    {"path": "QD-02-graffitis.jpg", "stop_id": "QD"},
    {"path": "TB-01.jpg", "stop_id": "TB"},
    {"path": "TA-02.jpg", "stop_id": "TA"},
    {"path": "UB-02-ice-snow.jpg", "stop_id": "UB"},
    {"path": "QC-02.jpg", "stop_id": "QC"},
    {"path": "PA-02-dirty.jpg", "stop_id": "PA"},
    {"path": "TD-02-dirty.jpg", "stop_id": "TD"},
    {"path": "RD-02.jpg", "stop_id": "RD"},
    {"path": "NC-01.jpg", "stop_id": "NC"},
]

upload_batch(
    batch=batch_3,
    source=SOURCE_PATH,
    target=TARGET_PATH,
    batch_number=3
)

Create new reports and update the incidents:

In [ ]:
run_bigquery(reports_creation_query)
run_bigquery(incidents_update_query)

In [ ]:
show_incidents()

## Batch 4

In [ ]:
batch_4 = [
    {"path": "SD-03.jpg", "stop_id": "SD"},
    {"path": "UB-02.jpg", "stop_id": "UB"},
    {"path": "PD-02.jpg", "stop_id": "PD"},
    {"path": "QC-01.jpg", "stop_id": "QC"},
    {"path": "RC-01.jpg", "stop_id": "RC"},
    {"path": "NA-02.jpg", "stop_id": "NA"},
    {"path": "PA-01.jpg", "stop_id": "PA"},
    {"path": "SA-02.jpg", "stop_id": "SA"},
    {"path": "RC-02.jpg", "stop_id": "RC"},
    {"path": "RD-01.jpg", "stop_id": "RD"},
    {"path": "SB-01.jpg", "stop_id": "SB"},
    {"path": "PD-02-broken-glass.jpg", "stop_id": "PD"},
    {"path": "NB-02.jpg", "stop_id": "NB"},
    {"path": "PD-01.jpg", "stop_id": "PD"},
    {"path": "QB-02.jpg", "stop_id": "QB"},
    {"path": "TB-02.jpg", "stop_id": "TB"},
]

upload_batch(
    batch=batch_4,
    source=SOURCE_PATH,
    target=TARGET_PATH,
    batch_number=4
)

In [ ]:
run_bigquery(reports_creation_query)
run_bigquery(incidents_update_query)
show_incidents()

## Batch 5

In [ ]:
batch_5 = [
    {"path": "PC-02.jpg", "stop_id": "PC"},
    {"path": "UC-01.jpg", "stop_id": "UC"},
    {"path": "SD-01.jpg", "stop_id": "SD"},
    {"path": "SA-01.jpg", "stop_id": "SA"},
    {"path": "ND-01.jpg", "stop_id": "ND"},
    {"path": "UD-02.jpg", "stop_id": "UD"},
    {"path": "SC-01.jpg", "stop_id": "SC"},
    {"path": "MB-02.jpg", "stop_id": "MB"},
    {"path": "TD-01.jpg", "stop_id": "TD"},
    {"path": "MD-01.jpg", "stop_id": "MD"},
    {"path": "RA-01.jpg", "stop_id": "RA"},
    {"path": "MC-01.jpg", "stop_id": "MC"},
    {"path": "RB-01.jpg", "stop_id": "RB"},
]

upload_batch(
    batch=batch_5,
    source=SOURCE_PATH,
    target=TARGET_PATH,
    batch_number=5
)

In [ ]:
run_bigquery(reports_creation_query)
run_bigquery(incidents_update_query)
show_incidents()